In [1]:
%matplotlib inline

import imp
import keras.backend
import keras.models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import pickle
import time
import keras

from keras.datasets import mnist
from keras.models import Model
from keras import optimizers

from matplotlib import cm, transforms

import innvestigate
import innvestigate.applications
import innvestigate.applications.mnist
import innvestigate.utils as iutils
import innvestigate.utils.visualizations as ivis
from innvestigate.utils.tests.networks import base as network_base

Using TensorFlow backend.
c:\program files\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\program files\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\program files\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\program files\python36\lib\site-packages\tensorflow\python\framework\dtype

In [2]:
%%bash
if [ ! -d "./stanfordSentimentTreebank" ]; then
    curl -L http://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip -O && unzip stanfordSentimentTreebank.zip
else
    echo "The data is already there. Skip downloading!!"
fi

Archive:  stanfordSentimentTreebank.zip
   creating: stanfordSentimentTreebank/
  inflating: stanfordSentimentTreebank/datasetSentences.txt  
   creating: __MACOSX/
   creating: __MACOSX/stanfordSentimentTreebank/
  inflating: __MACOSX/stanfordSentimentTreebank/._datasetSentences.txt  
  inflating: stanfordSentimentTreebank/datasetSplit.txt  
  inflating: __MACOSX/stanfordSentimentTreebank/._datasetSplit.txt  
  inflating: stanfordSentimentTreebank/dictionary.txt  
  inflating: __MACOSX/stanfordSentimentTreebank/._dictionary.txt  
  inflating: stanfordSentimentTreebank/original_rt_snippets.txt  
  inflating: __MACOSX/stanfordSentimentTreebank/._original_rt_snippets.txt  
  inflating: stanfordSentimentTreebank/README.txt  
  inflating: __MACOSX/stanfordSentimentTreebank/._README.txt  
  inflating: stanfordSentimentTreebank/sentiment_labels.txt  
  inflating: __MACOSX/stanfordSentimentTreebank/._sentiment_labels.txt  
  inflating: stanfordSentimentTreebank/SOStr.txt  
  inflating: stanfo

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0   329    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 6223k  100 6223k    0     0   975k      0  0:00:06  0:00:06 --:--:-- 1452k


In [3]:
%%bash
if [ ! -e "./stanfordSentimentTreebank/embeddings.npy" ]; then
    curl -L https://github.com/ArrasL/LRP_for_LSTM/raw/master/model/embeddings.npy -o stanfordSentimentTreebank/embeddings.npy &&
        curl -L https://github.com/ArrasL/LRP_for_LSTM/raw/master/model/vocab -o stanfordSentimentTreebank/vocab
else
    echo "The data is already there. Skip downloading!!"
fi

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   147  100   147    0     0    350      0 --:--:-- --:--:-- --:--:--   350
100 9158k  100 9158k    0     0  5176k      0  0:00:01  0:00:01 --:--:-- 11.2M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   138  100   138    0     0    542      0 --:--:-- --:--:-- --:--:--   545
100  207k  100  207k    0     0   430k      0 --:--:-- --:--:-- --:--:-- 8235k


In [4]:
!pip install ftfy

Defaulting to user installation because normal site-packages is not writeable


In [5]:
from ftfy import fix_encoding

In [6]:
DATA_PATH = './stanfordSentimentTreebank'

In [7]:
with open('%s/vocab' % DATA_PATH, 'rb') as f:
    vocabs = pickle.load(f) 
    total_vocabs = len(vocabs) 

    # Unknown vocabs are set to <UNK>.
    encoder = dict(zip(['<UNK>'] + vocabs, range(0, len(vocabs) +1)))
    decoder = dict(zip(encoder.values(), encoder.keys()))
    
    print('We have %d vocabs.' % len(encoder))

We have 19539 vocabs.


In [8]:
pretrained_embedding = np.load('%s/embeddings.npy' % DATA_PATH)

# Unknown vocabs will have embedding weights of zero.
embedding = np.zeros((pretrained_embedding.shape[0]+1, pretrained_embedding.shape[1]))
embedding[1:, :] = pretrained_embedding

In [9]:
# load all necessary files
df_reviews = pd.read_csv('%s/datasetSentences.txt' % DATA_PATH, sep='\t')

df_reviews['phase'] = df_reviews.sentence.apply(lambda s: fix_encoding(s))\
    .apply(lambda s: s.replace('-LRB-', '(').replace('-RRB-', ')'))

df_reviews['sostr'] = pd.read_csv('%s/SOStr.txt' % DATA_PATH,
                                  sep='\t',encoding='utf-8',
                                  header=None, names=['sostr']
                                 )

df_reviews['splitset_label'] = pd.read_csv('%s/datasetSplit.txt' % DATA_PATH,
                                           sep=',', header=0
                                          )['splitset_label']


df_phases = pd.read_csv('%s/dictionary.txt' % DATA_PATH,
                        sep='|', names=['phase', 'phase_id']
                       )

df_sentiment_labels = pd.read_csv('%s/sentiment_labels.txt' % DATA_PATH,
                                  sep='|', names=['phase_id', 'sentiment_value'],
                                  header=0
                                 )

df_reviews_with_sentiment_value = df_reviews.merge(df_phases, how='inner', on=['phase'])\
    .merge(df_sentiment_labels, on='phase_id')

df_reviews_with_sentiment_value[:5]

,sentence_index,sentence,phase,sostr,splitset_label,phase_id,sentiment_value
0,1,The Rock is destined to be the 21st Century 's...,The Rock is destined to be the 21st Century 's...,The|Rock|is|destined|to|be|the|21st|Century|'s...,1,226166,0.69444
1,2,The gorgeously elaborate continuation of `` Th...,The gorgeously elaborate continuation of `` Th...,The|gorgeously|elaborate|continuation|of|``|Th...,1,226300,0.83333
2,3,Effective but too-tepid biopic,Effective but too-tepid biopic,Effective|but|too-tepid|biopic,2,13995,0.51389
3,4,If you sometimes like to go to the movies to h...,If you sometimes like to go to the movies to h...,If|you|sometimes|like|to|go|to|the|movies|to|h...,2,14123,0.73611
4,5,"Emerges as something rare , an issue movie tha...","Emerges as something rare , an issue movie tha...","Emerges|as|something|rare|,|an|issue|movie|tha...",2,13999,0.86111


In [ ]:
def sentiment_discretizer(sentiment_value):
    if 0 <= sentiment_value <= 0.2:
        return 'very_negative'
    elif 0.2 < sentiment_value <= 0.4:
        return 'negative'
    elif 0.4 < sentiment_value <= 0.6:
        return 'neutral'
    elif 0.6 < sentiment_value <= 0.8:
        return 'positive'
    elif 0.8 < sentiment_value <= 1:
        return 'very_positive'
    
df_reviews_with_sentiment_value['label'] = df_reviews_with_sentiment_value.sentiment_value.apply(sentiment_discretizer)